### First, we need to import the GitbaseSessionBuilder and create a session

In [ ]:
import tech.sourced.gitbase.spark.GitbaseSessionBuilder

val spark = SparkSession.builder().appName("example")
    .master("local[*]")
    .config("spark.driver.host", "localhost")
    .registerGitbaseSource("gitbase:3306")
    .getOrCreate()

### Get all repositories

In [5]:
spark.sql("SELECT * FROM repositories").show()

+-----------------------+
|repository_id          |
+-----------------------+
|gitbase                |
|gitbase-spark-connector|
+-----------------------+



### Get HEAD references of repositories

In [8]:
spark.sql("SELECT * FROM refs WHERE ref_name = 'HEAD'").show(truncate=false)

+-----------------------+--------+----------------------------------------+
|repository_id          |ref_name|commit_hash                             |
+-----------------------+--------+----------------------------------------+
|gitbase-spark-connector|HEAD    |d9c0363978f5c869022dcf3266fd88fb2268c35a|
|gitbase                |HEAD    |38168fb16f0bf05286c631d4700272e92560f55a|
+-----------------------+--------+----------------------------------------+



### Get all the repositories where a specific user contributes on HEAD reference

In [10]:
spark.sql("""SELECT refs.repository_id
FROM refs
NATURAL JOIN commits
WHERE
    commits.commit_author_name = 'Javi Fontan'
    AND refs.ref_name = 'HEAD'""").show()

+-------------+
|repository_id|
+-------------+
|      gitbase|
+-------------+



### First commit on HEAD history for all repositories

In [13]:
spark.sql("""SELECT
    f.file_path,
    f.repository_id
FROM
    commit_files f
NATURAL JOIN
    ref_commits r
WHERE
    r.ref_name = 'HEAD'
    AND r.history_index = 0""").show(truncate=false)

+------------------------------------------------------------------+-----------------------+
|file_path                                                         |repository_id          |
+------------------------------------------------------------------+-----------------------+
|.gitignore                                                        |gitbase-spark-connector|
|.travis.yml                                                       |gitbase-spark-connector|
|Dockerfile                                                        |gitbase-spark-connector|
|LICENSE                                                           |gitbase-spark-connector|
|README.md                                                         |gitbase-spark-connector|
|build.sbt                                                         |gitbase-spark-connector|
|project/Dependencies.scala                                        |gitbase-spark-connector|
|project/build.properties                                          |gi

### Get the number of blobs per HEAD commit

In [24]:
spark.sql("""SELECT COUNT(commit_hash), commit_hash
FROM ref_commits as
NATURAL JOIN commits
NATURAL JOIN commit_blobs
WHERE ref_name = 'HEAD'
GROUP BY commit_hash""").show(truncate=false)

+------------------+----------------------------------------+
|count(commit_hash)|commit_hash                             |
+------------------+----------------------------------------+
|33                |478a42cd2a28b12ae3dbded699b02b39c12ebbc2|
|35                |63f9df67185e23387adcb41b7915cc3c738ac88d|
|2448              |15324804bfc5c8f788a400ee8d060a9841f7cd29|
|3586              |b31a60153f1bc36fa322c8ff684393733cb629f3|
|5919              |579894afe27ff6e2fc28b2fb63f1e34a272b6384|
|41                |2a41599d9cf2838636b82f5fe2fc89ea177f803a|
|2446              |9e3dd760bdf2f0bc54c3f45a0952ea80e72a75b2|
|6661              |8bd1d81a17625e1b4a537199151abcac20401407|
|4820              |a51b92224b79589a9015e4edb5b9f7dea974a6ac|
|5478              |c52fcf58ea0a4b3b5e0cfe0dbbe99c70f1dfa9a4|
|6256              |9259869628f1f54be6df34bafe3bc6a52796ec2e|
|4543              |d3db798e7f0ce18ea18cf532ee8a50e1cc7542bf|
|6609              |9ac9f0bb55a1fc806bc39f8d3bdff9d3c1927f8f|
|64     

### Get commits per committer, per month in 2018

In [23]:
spark.sql("""SELECT COUNT(*) as num_commits, month, repo_id, committer_email
FROM (
    SELECT
        MONTH(committer_when) as month,
        r.repository_id as repo_id,
        committer_email
    FROM ref_commits r
    INNER JOIN commits c
        ON YEAR(c.committer_when) = 2018
        AND r.repository_id = c.repository_id
        AND r.commit_hash = c.commit_hash
    WHERE r.ref_name = 'HEAD'
) as t
GROUP BY committer_email, month, repo_id""").show(truncate=false)

+-----------+-----+-----------------------+-------------------------------+
|num_commits|month|repo_id                |committer_email                |
+-----------+-----+-----------------------+-------------------------------+
|10         |9    |gitbase                |manu.carmona90@gmail.com       |
|4          |4    |gitbase                |manu.carmona90@gmail.com       |
|6          |10   |gitbase                |kuba@sourced.tech              |
|1          |10   |gitbase-spark-connector|coolakshat24@gmail.com         |
|14         |8    |gitbase                |manu.carmona90@gmail.com       |
|4          |5    |gitbase                |miguel@erizocosmi.co           |
|25         |7    |gitbase                |miguel@erizocosmi.co           |
|1          |4    |gitbase                |santi@mola.io                  |
|6          |5    |gitbase                |manu.carmona90@gmail.com       |
|9          |7    |gitbase                |jfontan@gmail.com              |
|4          

lastException: Throwable = null
